In [23]:
# import libraries and read excel file
import pandas as pd
!conda install -c anaconda xlrd --yes
Toronto_data = pd.read_excel('https://torontoproject202102.s3.eu-de.cloud-object-storage.appdomain.cloud/Toronto_postal_codes.xlsx',
                       sheet_name='Foglio1'
                      )


Solving environment: done

# All requested packages already installed.



In [25]:
#take a look at the data
Toronto_data.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [26]:
# skip not assigned boroughs:
TD = Toronto_data.drop(Toronto_data[(Toronto_data.Borough == "Not assigned")].index)

# give "Not assigned" Neighborhoods same name as Borough:
TD.Neighbourhood.replace("Not assigned", TD.Borough, inplace=True)

# copy Borough value to Neighborhood if NaN:
TD.Neighbourhood.fillna(TD.Borough, inplace=True)

# drop duplicate rows:
TD=TD.drop_duplicates()

# extract titles from columns
TD.update(
    TD.Neighbourhood.loc[
        lambda x: x.str.contains('title')
    ].str.extract('title=\"([^\"]*)',expand=False))

TD.update(
    TD.Borough.loc[
        lambda x: x.str.contains('title')
    ].str.extract('title=\"([^\"]*)',expand=False))

# delete Toronto annotation from Neighbourhood:
TD.update(
    TD.Neighbourhood.loc[
        lambda x: x.str.contains('Toronto')
    ].str.replace(", Toronto",""))
TD.update(
    TD.Neighbourhood.loc[
        lambda x: x.str.contains('Toronto')
    ].str.replace("\(Toronto\)",""))

TD1=TD.rename(columns={'Postal Code': 'Postcode'})

TD1.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [29]:
# combine multiple neighborhoods with the same post code
TD2 = pd.DataFrame({'Postcode':TD1.Postcode.unique()})
TD2['Borough']=pd.DataFrame(list(set(TD1['Borough'].loc[TD1['Postcode'] == x['Postcode']])) for i, x in TD2.iterrows())
TD2['Neighborhood']=pd.Series(list(set(TD1['Neighbourhood'].loc[TD1['Postcode'] == x['Postcode']])) for i, x in TD2.iterrows())
TD2['Neighborhood']=TD2['Neighborhood'].apply(lambda x: ', '.join(x))
TD2.dtypes

TD2.shape


(103, 3)